[Pythin quickstart for calendar APi](https://developers.google.com/workspace/calendar/api/quickstart/python)

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import os

def authenticate_calendar_api(credentials_path='credentials.json'):
    """
    Authenticate with Google Calendar API using service account credentials
    
    Args:
        credentials_path (str): Path to your credentials.json file
    
    Returns:
        googleapiclient.discovery.Resource: Authenticated Calendar service object
    """
    
    # Define the required scope for Google Calendar API
    SCOPES = ['https://www.googleapis.com/auth/calendar']
    
    try:
        # Load the credentials from the file
        credentials = service_account.Credentials.from_service_account_file(
            credentials_path,
            scopes=SCOPES
        )
        
        # Build the Calendar service
        service = build('calendar', 'v3', credentials=credentials)
        
        print("✅ Successfully authenticated with Google Calendar API")
        return service
        
    except Exception as e:
        print(f"❌ Error authenticating with Google Calendar API: {e}")
        return None

# Example usage:
if __name__ == "__main__":
    # Authenticate
    calendar_service = authenticate_calendar_api()
    
    if calendar_service:
        # Example: List calendars
        calendar_list = calendar_service.calendarList().list().execute()
        
        print("\n📅 Available Calendars:")
        for calendar in calendar_list.get('items', []):
            print(f"- {calendar['summary']} (ID: {calendar['id']})")